In [ ]:
#! conda install -y gdal

In [ ]:
import os
import boto3
import logging
import shutil
from datetime import datetime
import gdal

In [ ]:
input_dir  = 'tiles/'
output_dir = 'output/'

In [ ]:
try:
    os.mkdir(input_dir)
    os.mkdir(output_dir)
except OSError as error:
    print(error)

In [ ]:
#Processing each image and saving the results into the output folder
tiles_folders = os.listdir(input_dir)
print(tiles_folders)

for folder_name in tiles_folders:
    
    #Create a file list
    command = "ls %s > %s" % (input_dir + folder_name + '/*.tif', output_dir + folder_name + '/filelist.txt')
    print(command)
    os.system(command)

    #Create a .vrt file from the filelist
    command = "gdalbuildvrt -input_file_list %s %s" % (output_dir + folder_name + '/filelist.txt', output_dir + folder_name + '/merged.vrt')
    print(command)
    os.system(command)
    
    #Convert the .vrt file to GTiff
    command = "gdal_translate -of GTiff  %s %s" % (output_dir + folder_name + '/merged.vrt', output_dir + folder_name + '/merged.tif')
    print(command)
    os.system(command)

In [ ]:
#Upload the resutls to S3

folder_list = os.listdir(output_dir)
#print(folder_list)

for folder_name in folder_list:
    
    image_list = os.listdir(output_dir + folder_name)
    #print(image_list)
    
    for image_name in image_list:
        
        if image_name != 'merged.tif':
            continue
        
        upload_file_path = output_dir + folder_name + '/' + image_name
        
        print(upload_file_path)
        
        s3.upload_file(upload_file_path, 
                       s3_bucket_output, 
                       folder_name + '.tif')


In [ ]:
#Removing the files from the input and output directories
shutil.rmtree(input_dir)
shutil.rmtree(output_dir)
logger.info('Removed the temp folders.')